In [1]:
import os
import sys
import argparse

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import Sampler

import csv
import random
import numpy as np
import pandas as pd

In [2]:
from PIL import Image
filenameToPILImage = lambda x: Image.open(x)

In [3]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(SEED)
np.random.seed(SEED)
def worker_init_fn(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)

In [4]:

# mini-Imagenet dataset
class MiniDataset(Dataset):
    def __init__(self, csv_path, data_dir):
        self.data_dir = data_dir
        self.data_df = pd.read_csv(csv_path).set_index("id")

        self.transform = transforms.Compose([
            filenameToPILImage,
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

    def __getitem__(self, index):
        path = self.data_df.loc[index, "filename"]
        label = self.data_df.loc[index, "label"]
        image = self.transform(os.path.join(self.data_dir, path))
        return image, label

    def __len__(self):
        return len(self.data_df)

class GeneratorSampler(Sampler):
    def __init__(self, episode_file_path):
        episode_df = pd.read_csv(episode_file_path).set_index("episode_id")
        self.sampled_sequence = episode_df.values.flatten().tolist()

    def __iter__(self):
        return iter(self.sampled_sequence) 

    def __len__(self):
        return len(self.sampled_sequence)


In [5]:
test_dataset = MiniDataset('hw4_data\mini\\val.csv', 'hw4_data\mini\\val')
test_loader = DataLoader(
        test_dataset, batch_size=5 * (15+ 1),
        pin_memory=False, worker_init_fn=worker_init_fn,
        sampler=GeneratorSampler('hw4_data\mini\\val_testcase.csv'))


In [6]:
len(test_loader)

600

In [11]:
for i, (data, target) in enumerate(test_loader):
    if(i==1):break
    # split data into support and query data
    support_input = data[:5 * 1,:,:,:] 
    query_input   = data[5 * 1:,:,:,:]

    # create the relative label (0 ~ N_way-1) for query data
    label_encoder = {target[i *1] : i for i in range(5)}
    query_label = torch.cuda.LongTensor([label_encoder[class_name] for class_name in target[5 * 1:]])
    print(target[5:])
    print(label_encoder)
    print(query_label)

('n03535780', 'n03535780', 'n01855672', 'n02981792', 'n02981792', 'n02950826', 'n02174001', 'n01855672', 'n02981792', 'n02174001', 'n02174001', 'n01855672', 'n01855672', 'n02174001', 'n02950826', 'n03535780', 'n02174001', 'n02950826', 'n03535780', 'n02981792', 'n02950826', 'n02174001', 'n01855672', 'n03535780', 'n01855672', 'n02174001', 'n01855672', 'n01855672', 'n03535780', 'n02950826', 'n01855672', 'n03535780', 'n02950826', 'n02950826', 'n03535780', 'n02174001', 'n02174001', 'n01855672', 'n03535780', 'n02981792', 'n02950826', 'n02981792', 'n02950826', 'n02981792', 'n02950826', 'n03535780', 'n01855672', 'n02950826', 'n01855672', 'n02950826', 'n03535780', 'n02174001', 'n02981792', 'n01855672', 'n03535780', 'n02981792', 'n02981792', 'n03535780', 'n02174001', 'n02950826', 'n02174001', 'n03535780', 'n02981792', 'n03535780', 'n02950826', 'n02174001', 'n02174001', 'n01855672', 'n02981792', 'n02981792', 'n02950826', 'n01855672', 'n02981792', 'n02174001', 'n02981792')
{'n02981792': 0, 'n03535

In [25]:
a=torch.tensor([12,3,4,5])

In [27]:
a.to

[12, 3, 4, 5]

In [5]:
import torch
import torch.nn.functional as F

In [10]:
a = torch.randn((10,1600))
b = torch.randn((1,1600))

In [12]:
a = a[0]

In [13]:
F.cosine_similarity(b,a)

tensor([0.0303])